In [32]:
!pip install pyspark

  Using cached pyspark-3.5.4.tar.gz (317.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849827 sha256=53aba8c794699671c4aa54f858433e7230c4db44862166255fe1862346e36e79
  Stored in directory: /Users/m98095/Library/Caches/pip/wheels/8d/28/22/5dbae8a8714ef046cebd320d0ef7c92f5383903cf854c15c0c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9


In [33]:
!pip install setuptools

In [34]:
!pip show pyspark

Name: pyspark
Version: 3.5.4
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /Users/m98095/Documents/workshop_skatteetaten/.venv/lib/python3.11/site-packages
Requires: py4j
Required-by: 


In [41]:
# Initialize Spark session
spark = SparkSession.builder.appName("loan_acceptance").getOrCreate()

Could not find valid SPARK_HOME while searching ['/Users/m98095/Documents', '/Users/m98095/Documents/workshop_skatteetaten/.venv/lib/python3.13/site-packages/pyspark/spark-distribution', '/Users/m98095/Documents/workshop_skatteetaten/.venv/lib/python3.13/site-packages/pyspark', '/Users/m98095/Documents/workshop_skatteetaten/.venv/lib/python3.13/site-packages/pyspark/spark-distribution', '/Users/m98095/Documents/workshop_skatteetaten/.venv/lib/python3.13/site-packages/pyspark', '/Users/m98095/Documents/workshop_skatteetaten/.venv/lib/python3.13']


SystemExit: -1

In [ ]:
import numpy 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, Imputer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



# Read CSV file
df = spark.read.csv("./../../Downloads/loan_data.csv", header=True)

ModuleNotFoundError: No module named 'pyspark.ml'

In [ ]:

# impute rows with missing values
df = df.fillna(0)

# List of categorical columns to be encoded
categorical_columns = ["person_education", "person_home_ownership", "loan_intent", "previous_loan_defaults_on_file", "loan_status"]

# List of numerical columns
numerical_columns = ["person_age", "person_income", "person_emp_exp", "loan_amnt", "loan_int_rate", "loan_percent_income", "cb_person_cred_hist_length", "credit_score"]

for column in numerical_columns:
    df = df.withColumn(column, col(column).cast("float"))
    
# Create a StringIndexer and OneHotEncoder for each categorical column
indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=col + "_indexed", outputCol=col + "_encoded") for col in categorical_columns]

# Assemble all feature columns into a single vector
encoded_feature_columns = [col + "_encoded" for col in categorical_columns]
feature_columns = encoded_feature_columns + numerical_columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Create a StringIndexer for the target column
label_indexer = StringIndexer(inputCol="person_gender", outputCol="label")

# Combine all stages into a pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer])

# Fit and transform the data
df_preprocessed = pipeline.fit(df).transform(df)

# Split the data into training and test sets
train_df, test_df = df_preprocessed.randomSplit([0.8, 0.2], seed=42)

# Initialize the Random Forest classifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)

# Train the model
rf_model = rf.fit(train_df)

# Make predictions on the test set
predictions = rf_model.transform(test_df)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Test set accuracy: {accuracy}")
print(f"Test set precision: {precision}")
print(f"Test set recall: {recall}")
print(f"Test set F1-score: {f1}")


Test set accuracy: 0.5538153948683772
Test set precision: 0.30671149159321653
Test set recall: 0.5538153948683772
Test set F1-score: 0.39478498231663856


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Define the parameter grid
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [50, 100, 150])
             .addGrid(rf.maxDepth, [5, 10, 15])
             .build())

# Define the cross-validator
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

# Perform cross-validation
cv_model = crossval.fit(train_df)
cv_predictions = cv_model.transform(test_df)
cv_accuracy = evaluator.evaluate(cv_predictions)

print(f"Cross-validated accuracy: {cv_accuracy}")

25/02/11 09:40:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/02/11 09:40:13 WARN DAGScheduler: Broadcasting large task binary with size 1485.5 KiB
25/02/11 09:40:14 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
25/02/11 09:40:15 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
25/02/11 09:40:19 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
25/02/11 09:40:23 WARN DAGScheduler: Broadcasting large task binary with size 1485.5 KiB
25/02/11 09:40:24 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
25/02/11 09:40:26 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
25/02/11 09:40:27 WARN DAGScheduler: Broadcasting large task binary with size 4.9 MiB
25/02/11 09:40:29 WARN DAGScheduler: Broadcasting large task binary with size 7.1 MiB
25/02/11 09:40:30 WARN DAGSched

Cross-validated accuracy: 0.5538153948683772


In [ ]:


evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Test set accuracy: {accuracy}")
print(f"Test set precision: {precision}")
print(f"Test set recall: {recall}")
print(f"Test set F1-score: {f1}")

Test set accuracy: 0.5538153948683772
Test set precision: 0.30671149159321653
Test set recall: 0.5538153948683772
Test set F1-score: 0.39478498231663856


In [ ]:
df.show()

+----------+-------------+----------------+-------------+--------------+---------------------+---------+-----------------+-------------+-------------------+--------------------------+------------+------------------------------+-----------+
|person_age|person_gender|person_education|person_income|person_emp_exp|person_home_ownership|loan_amnt|      loan_intent|loan_int_rate|loan_percent_income|cb_person_cred_hist_length|credit_score|previous_loan_defaults_on_file|loan_status|
+----------+-------------+----------------+-------------+--------------+---------------------+---------+-----------------+-------------+-------------------+--------------------------+------------+------------------------------+-----------+
|      22.0|       female|          Master|      71948.0|           0.0|                 RENT|  35000.0|         PERSONAL|        16.02|               0.49|                       3.0|       561.0|                            No|          1|
|      21.0|       female|     High Scho

In [ ]:

# impute rows with missing values
df = df.fillna(0)

# List of categorical columns to be encoded
categorical_columns = ["person_education", "person_home_ownership", "loan_intent", "previous_loan_defaults_on_file", "person_gender"]

# List of numerical columns
numerical_columns = ["person_age", "person_income", "person_emp_exp", "loan_amnt", "loan_int_rate", "loan_percent_income", "cb_person_cred_hist_length", "credit_score"]

for column in numerical_columns:
    df = df.withColumn(column, col(column).cast("float"))
    
# Create a StringIndexer and OneHotEncoder for each categorical column
indexers = [StringIndexer(inputCol=col, outputCol=col + "_indexed") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=col + "_indexed", outputCol=col + "_encoded") for col in categorical_columns]

# Assemble all feature columns into a single vector
encoded_feature_columns = [col + "_encoded" for col in categorical_columns]
feature_columns = encoded_feature_columns + numerical_columns
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Create a StringIndexer for the target column
label_indexer = StringIndexer(inputCol="loan_status", outputCol="label")

# Combine all stages into a pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer])

# Fit and transform the data
df_preprocessed = pipeline.fit(df).transform(df)

# Split the data into training and test sets
train_df, test_df = df_preprocessed.randomSplit([0.8, 0.2], seed=42)

# Initialize the Random Forest classifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)

# Train the model
rf_model = rf.fit(train_df)

# Make predictions on the test set
predictions = rf_model.transform(test_df)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Test set accuracy: {accuracy}")
print(f"Test set precision: {precision}")
print(f"Test set recall: {recall}")
print(f"Test set F1-score: {f1}")


Test set accuracy: 0.8961457292013774
Test set precision: 0.9015613574798111
Test set recall: 0.8961457292013773
Test set F1-score: 0.8862195019192205


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Define the parameter grid
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [50, 100, 150])
             .addGrid(rf.maxDepth, [5, 10, 15])
             .build())

# Define the cross-validator
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

# Perform cross-validation
cv_model = crossval.fit(train_df)
cv_predictions = cv_model.transform(test_df)


cv_accuracy = evaluator.evaluate(cv_predictions)
precision = evaluator.evaluate(cv_predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(cv_predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(cv_predictions, {evaluator.metricName: "f1"})


print(f"Cross-validated accuracy: {cv_accuracy}")
print(f"CV set precision: {precision}")
print(f"CV set recall: {recall}")
print(f"CV set F1-score: {f1}")

25/02/11 10:28:21 WARN DAGScheduler: Broadcasting large task binary with size 1265.7 KiB
25/02/11 10:28:22 WARN DAGScheduler: Broadcasting large task binary with size 1893.1 KiB
25/02/11 10:28:23 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
25/02/11 10:28:25 WARN DAGScheduler: Broadcasting large task binary with size 1968.0 KiB
25/02/11 10:28:30 WARN DAGScheduler: Broadcasting large task binary with size 1265.7 KiB
25/02/11 10:28:31 WARN DAGScheduler: Broadcasting large task binary with size 1893.1 KiB
25/02/11 10:28:32 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
25/02/11 10:28:33 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
25/02/11 10:28:35 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
25/02/11 10:28:36 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB
25/02/11 10:28:39 WARN DAGScheduler: Broadcasting large task binary with size 9.0 MiB
25/02/11 10:28:41 WARN DAGScheduler: Br

Cross-validated accuracy: 0.9225813617682994
CV set precision: 0.9219400896797401
CV set recall: 0.9225813617682994
CV set F1-score: 0.9194192554156


In [ ]:


evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Test set accuracy: {accuracy}")
print(f"Test set precision: {precision}")
print(f"Test set recall: {recall}")
print(f"Test set F1-score: {f1}")

Test set accuracy: 0.9105853604354104
Test set precision: 0.9139568701590184
Test set recall: 0.9105853604354105
Test set F1-score: 0.9039385896929992
